In [18]:
import digitalhub as dh

project = dh.get_or_create_project("llm-ci")

In [19]:
llm_function = project.new_function("llm_classification",
                                    kind="huggingfaceserve",
                                    model_name="mymodel",
                                    path="huggingface://distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [20]:
llm_run = llm_function.run(action="serve", profile="template-a100")

Let us wait till the 'serve' is loaded compeletly

In [27]:
model_name = "mymodel"
json = {
    "inputs": [
        {
            "name": "input-0",
            "shape": [2],
            "datatype": "BYTES",
            "data": ["Hello, my dog is cute", "I am feeling sad"],
        },
    ]
}

In [28]:
llm_run.refresh()

{'kind': 'huggingfaceserve+run', 'metadata': {'project': 'llm-ci', 'name': '843d60b5-a1b7-491c-b8da-d71947dcc970', 'created': '2024-10-25T09:32:11.403Z', 'updated': '2024-10-25T09:34:18.789Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'huggingfaceserve+serve://llm-ci/llm_classification:6da5afcf-ee73-4142-a7cc-9c8cf4fdbfce', 'local_execution': False, 'function': 'huggingfaceserve://llm-ci/llm_classification:6da5afcf-ee73-4142-a7cc-9c8cf4fdbfce', 'profile': 'template-a100', 'path': 'huggingface://distilbert/distilbert-base-uncased-finetuned-sst-2-english', 'model_name': 'mymodel', 'service_type': 'NodePort'}, 'status': {'state': 'RUNNING', 'message': 'deployment d-huggingfaceserveserve-843d60b5-a1b7-491c-b8da-d71947dcc970 created, service s-huggingfaceserveserve-843d60b5-a1b7-491c-b8da-d71947dcc970 created', 'transitions': [{'status': 'RUNNING', 'message': 'deployment d-huggingfaceserveserve-843d60b5-a1b7-491c-b8da-d71947dcc970 created, service s

In [29]:
llm_run.invoke(model_name=model_name, json=json).json()

{'model_name': 'mymodel',
 'model_version': None,
 'id': '0c9834b3-9e0a-49ee-88e3-97e281e91198',
 'parameters': None,
 'outputs': [{'name': 'output-0',
   'shape': [2],
   'datatype': 'INT64',
   'parameters': None,
   'data': [1, 0]}]}

In [23]:
import os
new_folder = 'src'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

In [24]:
%%writefile "src/llm_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(path): 
    with pipeline_context() as pc:
         serve = pc.step(name="serve", function="llm_classification", action="serve", inputs={"path":path})

Overwriting src/llm_pipeline.py


In [25]:
workflow = project.new_workflow(name="pipeline_llm", kind="kfp", code_src= "src/llm_pipeline.py", handler = "myhandler")

In [8]:
workflow.run(parameters={"path": 'huggingface://distilbert/distilbert-base-uncased-finetuned-sst-2-english'})

{'kind': 'kfp+run', 'metadata': {'project': 'llm', 'name': '99fce4cc-5a12-4fa5-b846-9532c9e17a18', 'created': '2024-10-25T09:22:18.181Z', 'updated': '2024-10-25T09:22:18.201Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'kfp+pipeline://llm/pipeline_llm:067b4bb7-cae7-47cc-bf74-0e03e10576f0', 'local_execution': False, 'function': 'kfp://llm/pipeline_llm:067b4bb7-cae7-47cc-bf74-0e03e10576f0', 'source': {'source': 'src/llm_pipeline.py', 'handler': 'myhandler', 'base64': 'CmZyb20gZGlnaXRhbGh1Yl9ydW50aW1lX2tmcC5kc2wgaW1wb3J0IHBpcGVsaW5lX2NvbnRleHQKCmRlZiBteWhhbmRsZXIocGF0aCk6IAogICAgd2l0aCBwaXBlbGluZV9jb250ZXh0KCkgYXMgcGM6CiAgICAgICAgIHNlcnZlID0gcGMuc3RlcChuYW1lPSJzZXJ2ZSIsIGZ1bmN0aW9uPSJsbG1fY2xhc3NpZmljYXRpb24iLCBhY3Rpb249InNlcnZlIiwgaW5wdXRzPXsicGF0aCI6cGF0aH0pCg==', 'lang': 'python'}, 'inputs': {}, 'outputs': {}, 'parameters': {'path': 'huggingface://distilbert/distilbert-base-uncased-finetuned-sst-2-english'}}, 'status': {'state': 'READY', 'trans